In [1]:
!git clone https://github.com/elenadigrigoli/FDS-project.git

Cloning into 'FDS-project'...
remote: Enumerating objects: 43615, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 43615 (delta 45), reused 42 (delta 19), pack-reused 43538 (from 1)
Receiving objects: 100% (43615/43615), 1.22 GiB | 26.11 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Updating files: 100% (21723/21723), done.


In [13]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from skimage import io, color, transform
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, top_k_accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from PIL import Image
from preprocessing import *
from machine_learning import *
import cv2 as cv

### Load data

In [14]:
# Specify the base folder where all your image classes are stored
folder = 'data - Copia'

In [3]:
import random
for category in os.listdir(folder):
    count = 0
    delete = 0
    for image in os.listdir(folder + '/'  + category):
        count += 1
        while count > 1000:
            random_image = random.choice(os.listdir(folder + '/' + category)) 
            delete_image = folder + '/'  + category +  '/' + random_image
            os.remove(delete_image)
            delete+=1
            count-=1
           
    print(f'Deleted {delete} in {category}')

Deleted 500 in battery
Deleted 500 in biological
Deleted 810 in cardboard
Deleted 4323 in clothes
Deleted 2039 in glass
Deleted 500 in metal
Deleted 650 in paper
Deleted 915 in plastic
Deleted 977 in shoes
Deleted 500 in trash


### Load images in the right format for scikit-learn.

In [15]:
images, labels, class_names = load_images(folder, image_size=(224,224))
images = images.astype(np.float32)

In [ ]:
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from skimage import io, color
import numpy as np


# Compute HOG features
hog_features = []
for image in images:
    grayscale = color.rgb2gray(image)
    features = hog(grayscale, pixels_per_cell=(8, 8), cells_per_block=(2, 2), orientations=9)
    hog_features.append(features)

# Convert to numpy array
X = np.array(hog_features)
y = np.array(labels)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an SVM classifier
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
y_pred_train = clf.predict(X_train)
print(classification_report(y_train, y_pred_train))

## Preprocessing the images

### Image normalization

In [7]:
# Image normalization
images /= 255.0

### Remove noise from images using Gaussian Blur.

In [8]:
for i in range(len(images)):
    images[i] = cv.GaussianBlur(images[i], (5, 5), 0)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [11]:
# Appiattire ogni immagine in un vettore 1D
x_train = x_train.reshape(x_train.shape[0], -1)  # Output shape: (8000, 224*224*3)

### XGboost sulle immagini

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators = 200, learning_rate = 0.05, max_depth=6, objective="multi:softmax", reg_lambda = 5, reg_alpha = 0.5, random_state=42)
model.fit(x_train, y_train)

In [ ]:
y_pred_train = model.predict(x_train)
print(classification_report(y_train, y_pred_train))

In [ ]:
# Appiattire ogni immagine in un vettore 1D
x_test = x_test.reshape(x_test.shape[0], -1)  # Output shape: (8000, 224*224*3)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

In [12]:
rndf = RandomForestClassifier(n_estimators=150, max_depth=10, max_leaf_nodes=15, random_state=42)

rndf.fit(x_train, y_train)

KeyboardInterrupt: 

In [12]:
y_pred_train = rndf.predict(x_train)
print(classification_report(y_train, y_pred_train))

c:\Users\gabri\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] Impossibile trovare il file specificato
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\gabri\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\gabri\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gabri\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\gabri\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProce

: 

: 

In [ ]:
x_test = x_test.reshape(x_test.shape[0], -1)  # Output shape: (8000, 224*224*3)
y_pred = rndf.predict(x_test)
print(classification_report(y_test, y_pred))

## **Classification using color histograms**

### Compute color histograms and flatten images

In [8]:
feature_vectors = []
for img in images:  # 'images' contiene tutte le immagini
    feature_vector = calculate_histograms(img, norm=True)  # Calcola l'istogramma per l'immagine
    feature_vector /= feature_vector.sum()  # Normalizza l'istogramma al totale dei pixel
    feature_vectors.append(feature_vector)  # Aggiungi il vettore alla lista
features = np.array(feature_vectors)  # Converti in array numpy

### Train Test Split.

In [9]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [16]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators = 200, learning_rate = 0.05, max_depth=6, objective="multi:softmax", reg_lambda = 5, reg_alpha = 0.5, random_state=42)
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [17]:
y_pred_train = model.predict(x_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98       792
           1       0.96      0.99      0.98       798
           2       0.97      0.96      0.97       808
           3       0.97      0.98      0.97       799
           4       0.99      0.95      0.97       791
           5       0.95      0.96      0.95       814
           6       0.94      0.98      0.96       789
           7       0.98      0.95      0.97       796
           8       0.98      0.93      0.95       788
           9       0.97      0.97      0.97       825

    accuracy                           0.97      8000
   macro avg       0.97      0.97      0.97      8000
weighted avg       0.97      0.97      0.97      8000



In [18]:
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.66      0.64       208
           1       0.55      0.68      0.61       202
           2       0.63      0.47      0.54       192
           3       0.62      0.73      0.67       201
           4       0.59      0.43      0.49       209
           5       0.38      0.46      0.41       186
           6       0.55      0.61      0.58       211
           7       0.57      0.46      0.51       204
           8       0.44      0.32      0.37       212
           9       0.51      0.66      0.58       175

    accuracy                           0.55      2000
   macro avg       0.55      0.55      0.54      2000
weighted avg       0.55      0.55      0.54      2000



### SVM

In [ ]:
svm = SVC()
grid_svm = {'kernel':['poly', 'rbf'],
            'degree':[2, 3],
            'gamma': ['scale', 'auto']}

grid = GridSearchCV(svm, grid_svm, cv=5, scoring='accuracy', verbose=2)
if grid is not None:
    grid.fit(x_train, y_train)

    print(f'Best score: {grid.best_score_}')
    for hp, value in grid.best_params_.items():
        print(f'{hp}:{value}\n')

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END .................degree=2, gamma=scale, kernel=poly; total time= 2.2min
[CV] END .................degree=2, gamma=scale, kernel=poly; total time= 6.2min


In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf', C=1, gamma='scale')  # Usa kernel RBF per catturare relazioni non lineari
svm.fit(x_train, y_train)


### Training error for SVM

In [ ]:
y_pred_train = svm.predict(x_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.58      0.58      0.58      1229
           1       0.59      0.44      0.50      1209
           2       0.61      0.45      0.52      1421
           3       0.51      0.95      0.67      4253
           4       0.47      0.50      0.49      2402
           5       0.55      0.21      0.30      1210
           6       0.44      0.32      0.37      1317
           7       0.61      0.24      0.34      1526
           8       0.50      0.21      0.30      1581
           9       0.66      0.55      0.60      1223

    accuracy                           0.53     17371
   macro avg       0.55      0.44      0.47     17371
weighted avg       0.54      0.53      0.50     17371



### Test error for SVM

In [ ]:
y_pred = svm.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.49      0.47       271
           1       0.57      0.41      0.47       291
           2       0.56      0.32      0.41       389
           3       0.51      0.96      0.67      1070
           4       0.45      0.51      0.48       637
           5       0.37      0.13      0.20       290
           6       0.41      0.29      0.34       333
           7       0.54      0.22      0.31       389
           8       0.43      0.16      0.23       396
           9       0.44      0.41      0.42       277

    accuracy                           0.49      4343
   macro avg       0.47      0.39      0.40      4343
weighted avg       0.48      0.49      0.45      4343



### K-NN

In [ ]:
knn = KNeighborsClassifier()
grid_knn = {'n_neighbors':[i for i in range(50, 350, 50)],
            'weights':['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']}

grid = GridSearchCV(knn, grid_knn, cv=5, scoring='accuracy', verbose=2)
if grid is not None:
    grid.fit(x_train, y_train)

    print(f'Best score: {grid.best_score_}')
    for hp, value in grid.best_params_.items():
        print(f'{hp}:{value}\n')

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END ..metric=euclidean, n_neighbors=50, weights=uniform; total time=   0.9s
[CV] END ..metric=euclidean, n_neighbors=50, weights=uniform; total time=   0.8s
[CV] END ..metric=euclidean, n_neighbors=50, weights=uniform; total time=   0.8s
[CV] END ..metric=euclidean, n_neighbors=50, weights=uniform; total time=   0.9s
[CV] END ..metric=euclidean, n_neighbors=50, weights=uniform; total time=   1.0s
[CV] END .metric=euclidean, n_neighbors=50, weights=distance; total time=   0.8s
[CV] END .metric=euclidean, n_neighbors=50, weights=distance; total time=   0.9s
[CV] END .metric=euclidean, n_neighbors=50, weights=distance; total time=   1.0s
[CV] END .metric=euclidean, n_neighbors=50, weights=distance; total time=   0.9s
[CV] END .metric=euclidean, n_neighbors=50, weights=distance; total time=   0.8s
[CV] END .metric=euclidean, n_neighbors=100, weights=uniform; total time=   1.0s
[CV] END .metric=euclidean, n_neighbors=100, we

In [ ]:
knn = KNeighborsClassifier(n_neighbors=50, weights='distance', metric='manhattan', p=1)

knn.fit(x_train, y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=50, p=1,
                     weights='distance')

### Training error for KNN

In [ ]:
y_pred_train = knn.predict(x_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1229
           1       1.00      1.00      1.00      1209
           2       1.00      1.00      1.00      1421
           3       1.00      1.00      1.00      4253
           4       1.00      1.00      1.00      2402
           5       1.00      1.00      1.00      1210
           6       1.00      1.00      1.00      1317
           7       1.00      1.00      1.00      1526
           8       1.00      1.00      1.00      1581
           9       1.00      1.00      1.00      1223

    accuracy                           1.00     17371
   macro avg       1.00      1.00      1.00     17371
weighted avg       1.00      1.00      1.00     17371



### Test error for KNN

In [ ]:
y_pred = knn.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.39      0.46       271
           1       0.41      0.67      0.51       291
           2       0.61      0.30      0.41       389
           3       0.57      0.90      0.70      1070
           4       0.51      0.42      0.46       637
           5       0.40      0.17      0.24       290
           6       0.33      0.47      0.39       333
           7       0.62      0.23      0.33       389
           8       0.38      0.33      0.35       396
           9       0.56      0.36      0.44       277

    accuracy                           0.50      4343
   macro avg       0.50      0.42      0.43      4343
weighted avg       0.51      0.50      0.47      4343



### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()

grid_tree = {'criterion':['gini', 'entropy', 'log_loss'],
             'splitter' : ['best', 'random'],
             'max_depth': [5, 10, 15]}

grid = GridSearchCV(tree, grid_tree, cv=5, scoring='accuracy', verbose=2)
if grid is not None:
    grid.fit(x_train, y_train)

    print(f'Best score: {grid.best_score_}')
    for hp, value in grid.best_params_.items():
        print(f'{hp}:{value}\n')

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END .........criterion=gini, max_depth=5, splitter=best; total time=   3.9s
[CV] END .........criterion=gini, max_depth=5, splitter=best; total time=   3.8s
[CV] END .........criterion=gini, max_depth=5, splitter=best; total time=   4.1s
[CV] END .........criterion=gini, max_depth=5, splitter=best; total time=   3.8s
[CV] END .........criterion=gini, max_depth=5, splitter=best; total time=   3.9s
[CV] END .......criterion=gini, max_depth=5, splitter=random; total time=   0.4s
[CV] END .......criterion=gini, max_depth=5, splitter=random; total time=   0.4s
[CV] END .......criterion=gini, max_depth=5, splitter=random; total time=   0.3s
[CV] END .......criterion=gini, max_depth=5, splitter=random; total time=   0.5s
[CV] END .......criterion=gini, max_depth=5, splitter=random; total time=   0.4s
[CV] END ........criterion=gini, max_depth=10, splitter=best; total time=   7.7s
[CV] END ........criterion=gini, max_depth=10, s

In [ ]:
tree = DecisionTreeClassifier(criterion='log_loss', max_depth=10, splitter='best')
tree.fit(x_train, y_train)

DecisionTreeClassifier(criterion='log_loss', max_depth=10)

### Train error

In [ ]:
y_pred_train = tree.predict(x_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.65      0.60      0.63      1229
           1       0.66      0.66      0.66      1209
           2       0.63      0.55      0.59      1421
           3       0.62      0.86      0.72      4253
           4       0.56      0.62      0.59      2402
           5       0.52      0.32      0.40      1210
           6       0.48      0.50      0.49      1317
           7       0.66      0.50      0.57      1526
           8       0.60      0.39      0.47      1581
           9       0.84      0.66      0.74      1223

    accuracy                           0.62     17371
   macro avg       0.62      0.57      0.59     17371
weighted avg       0.62      0.62      0.61     17371



### Test error

In [ ]:
y_pred = tree.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.34      0.35      0.35       271
           1       0.47      0.48      0.47       291
           2       0.40      0.28      0.33       389
           3       0.57      0.77      0.65      1070
           4       0.36      0.40      0.38       637
           5       0.24      0.16      0.20       290
           6       0.28      0.28      0.28       333
           7       0.36      0.30      0.33       389
           8       0.29      0.20      0.23       396
           9       0.53      0.38      0.44       277

    accuracy                           0.43      4343
   macro avg       0.38      0.36      0.37      4343
weighted avg       0.41      0.43      0.41      4343



### RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd = RandomForestClassifier()
grid_rnd = {'n_estimators':[i for i in range(10, 60, 10)],
            'max_depth':[j for j in range(5, 20, 5)],
            'max_leaf_nodes': [5, 10, 15]}

grid = GridSearchCV(rnd, grid_rnd, cv=5, scoring='accuracy', verbose=2)
if grid is not None:
    grid.fit(x_train, y_train)

    print(f'Best score: {grid.best_score_}')
    for hp, value in grid.best_params_.items():
        print(f'{hp}:{value}\n')

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=10; total time=   0.6s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=10; total time=   0.6s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=10; total time=   0.5s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=10; total time=   0.6s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=10; total time=   0.6s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=20; total time=   1.2s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=20; total time=   1.2s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=20; total time=   1.2s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=20; total time=   1.2s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=20; total time=   1.2s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=30; total time=   1.9s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_

In [ ]:
rnd = RandomForestClassifier(n_estimators=50, max_depth=15, max_leaf_nodes=15, random_state=42)
rnd.fit(x_train, y_train)

RandomForestClassifier(max_depth=15, max_leaf_nodes=15, n_estimators=50,
                       random_state=42)

### Training error

In [ ]:
y_pred_train = rnd.predict(x_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.39      0.44      0.41      1229
           1       0.41      0.50      0.45      1209
           2       0.32      0.36      0.34      1421
           3       0.44      0.95      0.60      4253
           4       0.32      0.24      0.27      2402
           5       0.00      0.00      0.00      1210
           6       0.80      0.01      0.01      1317
           7       0.50      0.07      0.13      1526
           8       0.43      0.01      0.02      1581
           9       0.43      0.57      0.49      1223

    accuracy                           0.41     17371
   macro avg       0.40      0.32      0.27     17371
weighted avg       0.41      0.41      0.32     17371



c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Test error

In [ ]:
y_pred = rnd.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.35      0.43      0.38       271
           1       0.44      0.55      0.49       291
           2       0.30      0.30      0.30       389
           3       0.44      0.96      0.60      1070
           4       0.41      0.30      0.34       637
           5       0.00      0.00      0.00       290
           6       1.00      0.01      0.02       333
           7       0.67      0.12      0.20       389
           8       0.23      0.01      0.01       396
           9       0.43      0.57      0.49       277

    accuracy                           0.42      4343
   macro avg       0.43      0.32      0.28      4343
weighted avg       0.43      0.42      0.33      4343



c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **Classification with images as matrices** (size 300x300)

### Train, test split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
x_train = x_train.reshape(x_train.shape[0], -1)

### Random Forest

In [ ]:
rndf = RandomForestClassifier(n_estimators=50, max_depth=15, max_leaf_nodes=15, random_state=42)
rndf.fit(x_train, y_train)

RandomForestClassifier(max_depth=15, max_leaf_nodes=15, n_estimators=50,
                       random_state=42)

### Train error

In [ ]:
y_pred_train = rndf.predict(x_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.51      0.41      0.45      1229
           1       0.30      0.29      0.29      1209
           2       0.46      0.35      0.40      1421
           3       0.48      0.97      0.65      4253
           4       0.28      0.44      0.34      2402
           5       0.50      0.00      0.00      1210
           6       0.00      0.00      0.00      1317
           7       0.00      0.00      0.00      1526
           8       0.53      0.01      0.01      1581
           9       0.44      0.62      0.52      1223

    accuracy                           0.42     17371
   macro avg       0.35      0.31      0.27     17371
weighted avg       0.36      0.42      0.33     17371



c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Test error

In [ ]:
x_test = x_test.reshape(x_test.shape[0], -1)
y_pred = rndf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.39      0.42       271
           1       0.25      0.22      0.23       291
           2       0.43      0.28      0.34       389
           3       0.47      0.97      0.63      1070
           4       0.29      0.44      0.35       637
           5       0.00      0.00      0.00       290
           6       0.00      0.00      0.00       333
           7       0.00      0.00      0.00       389
           8       0.80      0.01      0.02       396
           9       0.39      0.57      0.46       277

    accuracy                           0.40      4343
   macro avg       0.31      0.29      0.25      4343
weighted avg       0.34      0.40      0.31      4343



c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Logistic Regression

In [ ]:
lr = LogisticRegression(multi_class='multinomial', solver = 'saga', penalty = 'elasticnet', l1_ratio=0.5, random_state=42)
lr.fit(x_train, y_train)

c:\Users\gabri\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


### Train error

In [ ]:
y_pred_train = lr.predict(x_train)
print(classification_report(y_train, y_pred_train))

### Test error

In [ ]:
y_pred = lr.predict(x_test)
print(classification_report(y_test, y_pred))

## Preprocessing with Autoencoders

In [3]:
# Trasformazioni delle immagini
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ridimensiona le immagini
    transforms.ToTensor(),         # Converti in tensore
])

# Carica il dataset
dataset_path = "/content/FDS-project/data"
train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Crea un DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Visualizza le classi
print(train_dataset.classes)

['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']


In [5]:
# Percorso alla directory principale del dataset
dataset_path = "/content/FDS-project/data"

# Crea DataLoader per train, validation e test set
train_loader, val_loader, test_loader = create_dataloaders(
    dataset_path=dataset_path,
    train_split=0.8,  # 70% per il training
    val_split=0.0,    # 20% per la validazione
    test_split=0.2,   # 10% per il test
    batch_size=32     # Dimensione del batch
)

# Verifica il numero di campioni
print(f"Train set size: {len(train_loader.dataset)}")
print(f"Validation set size: {len(val_loader.dataset)}")
print(f"Test set size: {len(test_loader.dataset)}")

Train set size: 17371
Validation set size: 0
Test set size: 4343


In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ridimensiona le immagini
    transforms.ToTensor(),         # Converte in tensori
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalizzazione
])

In [ ]:
# Sparse Autoencoder
class SparseAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Linear(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        # Flatten l'immagine
        x = x.view(x.size(0), -1)
        encoded = torch.relu(self.encoder(x))
        decoded = torch.sigmoid(self.decoder(encoded))
        return encoded, decoded

# Dimensione dell'input
input_size = 3 * 224 * 224  # Aggiorna in base alla risoluzione delle immagini
hidden_size = 64  # Numero di neuroni nascosti

autoencoder = SparseAutoencoder(input_size, hidden_size)

In [9]:
train_features, train_labels = extract_features(autoencoder, train_loader)
test_features, test_labels = extract_features(autoencoder, test_loader)

In [ ]:
rnd = RandomForestClassifier()
grid_rnd = {'n_estimators':[i for i in range(50, 250, 50)],
            'max_depth':[j for j in range(5, 20, 5)],
            'max_leaf_nodes': [5, 10, 15]}

grid = GridSearchCV(rnd, grid_rnd, cv=5, scoring='accuracy', verbose=2)
if grid is not None:
    grid.fit(train_features, train_labels)

    print(f'Best score: {grid.best_score_}')
    for hp, value in grid.best_params_.items():
        print(f'{hp}:{value}\n')

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=50; total time=   2.5s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=50; total time=   1.4s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=50; total time=   1.1s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=50; total time=   1.1s
[CV] END .....max_depth=5, max_leaf_nodes=5, n_estimators=50; total time=   1.1s
[CV] END ....max_depth=5, max_leaf_nodes=5, n_estimators=100; total time=   2.1s
[CV] END ....max_depth=5, max_leaf_nodes=5, n_estimators=100; total time=   2.1s
[CV] END ....max_depth=5, max_leaf_nodes=5, n_estimators=100; total time=   2.8s
[CV] END ....max_depth=5, max_leaf_nodes=5, n_estimators=100; total time=   2.5s
[CV] END ....max_depth=5, max_leaf_nodes=5, n_estimators=100; total time=   2.1s
[CV] END ....max_depth=5, max_leaf_nodes=5, n_estimators=150; total time=   3.2s
[CV] END ....max_depth=5, max_leaf_nodes=5, n_e

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best score: 0.3500655475619504
max_depth:15

max_leaf_nodes:15

n_estimators:50



In [ ]:
rnd = RandomForestClassifier(n_estimators=100, random_state=42)

rnd.fit(train_features, train_labels)

RandomForestClassifier(random_state=42)

### Train error

In [ ]:
labels_pred_train = rnd.predict(train_features)
print(classification_report(train_labels, labels_pred_train))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1195
           1       1.00      1.00      1.00      1213
           2       1.00      1.00      1.00      1461
           3       1.00      1.00      1.00      4261
           4       1.00      1.00      1.00      2428
           5       1.00      1.00      1.00      1210
           6       1.00      1.00      1.00      1319
           7       1.00      1.00      1.00      1516
           8       1.00      1.00      1.00      1589
           9       1.00      1.00      1.00      1179

    accuracy                           1.00     17371
   macro avg       1.00      1.00      1.00     17371
weighted avg       1.00      1.00      1.00     17371



### Test error

In [ ]:
labels_pred = rnd.predict(test_features)
print(classification_report(test_labels, labels_pred))

              precision    recall  f1-score   support

           0       0.54      0.40      0.46       305
           1       0.49      0.32      0.39       287
           2       0.60      0.52      0.56       349
           3       0.53      0.94      0.68      1062
           4       0.51      0.56      0.53       611
           5       0.43      0.13      0.20       290
           6       0.68      0.43      0.52       331
           7       0.67      0.40      0.50       399
           8       0.38      0.20      0.26       388
           9       0.55      0.55      0.55       321

    accuracy                           0.54      4343
   macro avg       0.54      0.45      0.47      4343
weighted avg       0.54      0.54      0.51      4343



### SVM

### Valutare se farlo, ci vuole un po' di tempo

In [12]:
svm = SVC()
grid_svm = {'kernel':['linear', 'poly'],
            'degree':[1, 2, 3],
            'tol': [1e-3, 1e-4]}

grid = GridSearchCV(svm, grid_svm, cv=5, scoring='accuracy', verbose=2)
if grid is not None:
    grid.fit(train_features, train_labels)

    print(f'Best score: {grid.best_score_}')
    for hp, value in grid.best_params_.items():
        print(f'{hp}:{value}\n')

Fitting 5 folds for each of 12 candidates, totalling 60 fits


KeyboardInterrupt: 

In [21]:
svm = SVC(kernel='poly', degree = 2, tol=1e-5, random_state=42)
svm.fit(train_features, train_labels)

SVC(degree=2, kernel='poly', random_state=42, tol=1e-05)

### Train error

In [22]:
labels_pred_train = svm.predict(train_features)
print(classification_report(train_labels, labels_pred_train))

              precision    recall  f1-score   support

           0       0.69      0.59      0.64      1184
           1       0.64      0.56      0.60      1215
           2       0.59      0.49      0.54      1450
           3       0.57      0.94      0.71      4249
           4       0.54      0.52      0.53      2414
           5       0.52      0.25      0.34      1184
           6       0.56      0.34      0.42      1329
           7       0.47      0.36      0.41      1545
           8       0.60      0.37      0.46      1584
           9       0.58      0.59      0.58      1217

    accuracy                           0.57     17371
   macro avg       0.58      0.50      0.52     17371
weighted avg       0.57      0.57      0.55     17371



### Test error

In [23]:
labels_pred = svm.predict(test_features)
print(classification_report(test_labels, labels_pred))

              precision    recall  f1-score   support

           0       0.63      0.49      0.56       316
           1       0.52      0.49      0.50       285
           2       0.47      0.36      0.41       360
           3       0.56      0.93      0.70      1074
           4       0.45      0.41      0.43       625
           5       0.31      0.14      0.19       316
           6       0.36      0.22      0.27       321
           7       0.39      0.33      0.36       370
           8       0.42      0.24      0.31       393
           9       0.49      0.52      0.51       283

    accuracy                           0.50      4343
   macro avg       0.46      0.41      0.42      4343
weighted avg       0.47      0.50      0.47      4343

